In [1]:
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
import pandas as pd
from math import e,sqrt
from scipy.optimize import least_squares
from scipy.optimize import leastsq

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor


In [2]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
df = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#df = df.loc[0: : 10]
data = data_set_util.clean_dataset(df)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503905,1.608867,0.003217,0.000033,0.114300,0.623283,0.041817,0.005217,0.008350,0.013433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503906,1.601233,0.003183,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503907,1.599333,0.003233,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503908,1.924267,0.003217,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [5]:
# MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
#                                                      max_leaf_nodes=10,
#                                                      min_samples_leaf=6,
#                                                      n_estimators=200))

def create_and_fit_model(X_train,y_train):
    #clf = MultiOutputRegressor(RandomForestRegressor(max_depth=2, random_state=0))
    clf = MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=15,
                                                     max_leaf_nodes=20,
                                                     min_samples_leaf=3,
                                                     n_estimators=200))
    clf.fit(X_train, y_train)
    
    return clf

In [6]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [7]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            
#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            Xtrain = scaler.fit_transform(ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'use [kW](t)':'precipProbability(t)'])
            Xtest = scaler.transform(ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'use [kW](t)':'precipProbability(t)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ttest.loc[:,'use [kW](t)':'precipProbability(t)'].columns
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'use [kW](t)':'precipProbability(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [8]:
n_windows = 30
train_size = 0.75 

model_rf = sliding_window(data,n_windows,train_size)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 16796


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 33592


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 470288


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 487084


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

In [9]:
df_forecats_columns = data.loc[:,'use [kW](t)':'precipProbability(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_rf.to_csv (r'all_windows_random_forest_kaggle_sh.csv', index = False, header=True)

measures = model_rf
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)


    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [10]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,use [kW](t),0.335,0.144,81.463,0.765,9.345,0.050,0.186,0.081,86.372,0.132,2.849,0.014
1,gen [kW](t),0.022,0.008,inf,0.954,9.524,0.045,0.009,0.004,NaN,0.036,3.002,0.017
2,Dishwasher [kW](t),0.054,0.007,inf,-1.503,66.661,0.039,0.015,0.003,NaN,13.212,15.974,0.011
3,Furnace 1 [kW](t),0.053,0.018,26.653,0.806,7.255,0.064,0.024,0.014,23.269,0.152,5.293,0.035
4,Furnace 2 [kW](t),0.047,0.019,14.125,0.734,4.889,0.085,0.034,0.018,11.920,0.785,3.662,0.044
5,Home office [kW](t),0.016,0.005,4.875,0.900,2.305,0.037,0.007,0.003,1.634,0.225,0.764,0.035
6,Fridge [kW](t),0.022,0.007,44.720,0.913,12.574,0.039,0.003,0.001,7.168,0.013,1.733,0.014
7,Wine cellar [kW](t),0.012,0.003,11.385,0.934,4.227,0.052,0.018,0.003,11.008,0.127,2.850,0.023
8,Garage door [kW](t),0.012,0.001,7.265,0.130,2.183,0.033,0.013,0.001,12.912,0.817,0.761,0.011
9,Kitchen 12 [kW](t),0.008,0.002,inf,-46.999,7.544,0.248,0.015,0.005,NaN,109.590,7.322,0.376


In [12]:
final_measures.to_csv (r'random_forest_kaggle_sh_with_std.csv', index = False, header=True)

In [53]:
# y_p=clf.predict(df_testx)
# columns=df_original.columns
# y_p=pd.DataFrame(y_p,columns=columns)
# y_p

In [54]:
# y_p.shape,df_original.shape

In [55]:

# col=df_original.columns
# result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]
#     }
    
# final_result = {
        
#          "rmse": [],
         
#          "mae": [],
#          "r2": [],
#          "mape":[],
#          "variable":[]}
# for col in col: 
#   original = df_original[col].values
#   forecast = y_p[col].values
#   mae = round(mean_absolute_error(original,forecast),3)
#   r2 = round(r2_score(original,forecast),3)
#   rmse = round(mean_squared_error(original,forecast,squared=False),3)             
#   mape = round(mean_absolute_percentage_error(original,forecast),3)

#   result["rmse"].append(rmse)
#   result["mae"].append(mae)
#   result["r2"].append(r2)
#   result["mape"].append(mape)
  
#   result["variable"].append(col)
        
# measures = pd.DataFrame(result)

In [57]:

# import statistics
# final_result = {
#     "variable": [],
#     "rmse": [],
#     # "nrmse": [],
#     "mae": [],
#     "r2": [],
#     "mape":[]
# }


# var = measures.groupby("variable")

# for col in columns:
    
#     var_agr = var.get_group(col)
           
#     rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
#     # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
#     mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
#     r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
#     mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

#     final_result["variable"].append(col)
#     final_result["rmse"].append(rmse)
#     # final_result["nrmse"].append(nrmse)
#     final_result["mae"].append(mae)
#     final_result["r2"].append(r2)
#     final_result["mape"].append(mape)
        
#     print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
# final_measures_kpca = pd.DataFrame(final_result) 

# print("Statistics MIMO (test): ")
# final_measures_kpca

## RandomizedSearchCV

In [32]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__criterion=['friedman_mse', 'mse'], estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=list(range(2,30)), estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 300, 'estimator__min_samples_split': 10, 'estimator__min_samples_leaf': 10, 'estimator__max_leaf_nodes': 10, 'estimator__max_depth': 11, 'estimator__criterion': 'friedman_mse'}


In [79]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     max_leaf_nodes=10,
                                                     min_samples_leaf=6,
                                                     n_estimators=200))

In [36]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,52.135333,0.614335,20.820613,35.637159,17.464842,39.345009,20.094992,36.306186,19.686600,35.127142,...,23.234457,40.318124,19.127906,39.529253,1.947604,767.483022,90.341060,2.055098,62.000253,0.297794
1,102.043481,8.526935,21.753013,51.719449,21.677506,45.033281,21.453808,45.635964,20.789516,40.907764,...,20.466197,47.142882,15.974800,43.407486,2.303926,753.761075,90.341060,3.035610,26.810029,1.446189
2,52.135333,0.614335,21.527538,35.649144,17.455535,41.189242,21.453808,34.820087,20.146586,33.406486,...,22.257965,40.318124,19.555863,38.014425,-0.186586,767.472451,84.652585,2.055098,62.000253,-1.395935
3,52.135333,0.614335,21.527538,41.826678,23.244487,34.649029,22.290282,36.306186,22.702350,35.127142,...,21.461350,38.346312,20.294869,38.030387,9.444514,767.472451,71.753596,5.148894,39.822617,3.708089
4,75.118513,1.315244,20.006884,35.552055,16.889650,39.452734,19.977346,36.306186,19.409728,35.127142,...,22.257965,40.318124,19.122853,41.444712,-0.524435,767.483022,95.902044,4.096666,49.312474,-1.367976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,57.614808,0.614335,25.059279,46.703391,22.985673,49.248262,26.425289,44.024347,25.162302,45.147622,...,25.828000,49.834027,23.931157,47.390389,12.260373,743.983451,96.012178,1.091764,60.778250,13.250582
4930,57.614808,0.614335,20.814933,40.054492,17.455535,46.284507,20.651397,38.187678,20.789516,35.130568,...,22.257965,42.771625,19.555863,39.529253,4.659642,763.546792,90.306652,4.096666,54.493599,2.675181
4931,97.730313,5.963148,20.820613,38.223944,19.523836,37.180729,19.977346,38.187678,21.979011,35.180663,...,21.461350,36.939784,18.361181,41.444712,6.900888,749.056157,78.572028,5.170294,39.822617,2.670020
4932,97.730313,6.838882,23.003120,39.328219,22.701635,37.467221,21.453808,42.322182,21.742429,43.017442,...,20.466197,44.920472,17.701398,44.594727,2.303926,759.707522,96.012178,3.035610,34.652202,1.541970


In [37]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [38]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 64.672, 28.953, 0.588, 0.292)
Results: ('lights(t)', 4.647, 2.543, 0.642, 4716657651680834.0)
Results: ('T1(t)', 0.208, 0.169, 0.983, 0.008)
Results: ('RH_1(t)', 0.651, 0.435, 0.973, 0.011)
Results: ('T2(t)', 0.289, 0.222, 0.982, 0.011)
Results: ('RH_2(t)', 0.608, 0.466, 0.977, 0.012)
Results: ('T3(t)', 0.262, 0.21, 0.983, 0.01)
Results: ('RH_3(t)', 0.452, 0.347, 0.981, 0.009)
Results: ('T4(t)', 0.263, 0.206, 0.983, 0.01)
Results: ('RH_4(t)', 0.533, 0.424, 0.985, 0.011)
Results: ('T5(t)', 0.235, 0.173, 0.983, 0.009)
Results: ('RH_5(t)', 2.442, 1.043, 0.928, 0.019)
Results: ('T6(t)', 0.812, 0.667, 0.982, 3522085035969.868)
Results: ('RH_6(t)', 2.781, 2.243, 0.992, 0.189)
Results: ('T7(t)', 0.264, 0.204, 0.984, 0.01)
Results: ('RH_7(t)', 0.67, 0.561, 0.983, 0.016)
Results: ('T8(t)', 0.246, 0.197, 0.984, 0.009)
Results: ('RH_8(t)', 0.705, 0.565, 0.982, 0.013)
Results: ('T9(t)', 0.226, 0.171, 0.987, 0.009)
Results: ('RH_9(t)', 0.564, 0.445, 0.981, 0.011)
Results:

,variable,rmse,mae,r2,mape
0,Appliances(t),64.672,28.953,0.588,2.920000e-01
1,lights(t),4.647,2.543,0.642,4.716658e+15
2,T1(t),0.208,0.169,0.983,8.000000e-03
3,RH_1(t),0.651,0.435,0.973,1.100000e-02
4,T2(t),0.289,0.222,0.982,1.100000e-02
5,RH_2(t),0.608,0.466,0.977,1.200000e-02
6,T3(t),0.262,0.210,0.983,1.000000e-02
7,RH_3(t),0.452,0.347,0.981,9.000000e-03
8,T4(t),0.263,0.206,0.983,1.000000e-02
9,RH_4(t),0.533,0.424,0.985,1.100000e-02


## MinMaxScaler

In [40]:
x,y=data.loc[:,:],data.loc[:,:]

In [41]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [43]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__n_estimators=[20, 50, 100, 200, 300],
                     estimator__min_samples_split=list(range(2,11)),
                     estimator__max_depth=[2,5,10,15,20,30], estimator__min_samples_leaf=list(range(1,11)),
                     estimator__max_leaf_nodes=[5, 10, 20, 30, 50])

clf=RandomForestRegressor()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=5)

hyperparameters_tuning = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__n_estimators': 200, 'estimator__min_samples_split': 2, 'estimator__min_samples_leaf': 6, 'estimator__max_leaf_nodes': 10, 'estimator__max_depth': 20}


In [71]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     max_leaf_nodes=10,
                                                     min_samples_leaf=6,
                                                     n_estimators=200))

In [44]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,0.039434,0.008902,0.425789,0.235590,0.099039,0.530996,0.240240,0.353788,0.411860,0.295773,...,0.632443,0.367415,0.442039,0.429146,0.223862,0.887851,0.873308,0.152202,0.938817,0.312147
1,0.086225,0.122735,0.525106,0.678022,0.406650,0.691727,0.353591,0.791135,0.512900,0.550553,...,0.380865,0.601613,0.112297,0.590016,0.235508,0.568367,0.873308,0.224891,0.397525,0.364296
2,0.039434,0.008902,0.499981,0.235842,0.098710,0.583193,0.353591,0.283310,0.455532,0.219933,...,0.545057,0.367415,0.485151,0.366260,0.155059,0.887851,0.798282,0.152202,0.938817,0.237269
3,0.039434,0.008902,0.499981,0.407511,0.515572,0.398840,0.423065,0.353788,0.683338,0.295773,...,0.471599,0.300546,0.562447,0.366910,0.464825,0.887851,0.629079,0.381375,0.597247,0.468121
4,0.059796,0.018568,0.339295,0.234520,0.059409,0.534275,0.230826,0.353788,0.388446,0.295773,...,0.545057,0.367415,0.441244,0.508506,0.144079,0.887851,0.945928,0.303419,0.743709,0.237829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,0.045054,0.008902,0.873579,0.546167,0.500959,0.808855,0.766842,0.715514,0.907035,0.738807,...,0.871277,0.694384,0.940637,0.753967,0.554356,0.341180,0.947617,0.080900,0.919292,0.902506
4930,0.045054,0.008902,0.424897,0.359298,0.098710,0.725992,0.286898,0.440899,0.512900,0.295773,...,0.545057,0.451559,0.485151,0.429146,0.310289,0.796748,0.873308,0.303419,0.821006,0.422924
4931,0.081724,0.086832,0.426557,0.307917,0.249033,0.470143,0.230826,0.440899,0.619854,0.297416,...,0.471599,0.250731,0.362180,0.508506,0.382494,0.459455,0.718524,0.382445,0.597247,0.422924
4932,0.081724,0.101966,0.656019,0.338444,0.477228,0.476642,0.353591,0.635665,0.598661,0.643446,...,0.380865,0.525309,0.292563,0.634128,0.235508,0.707416,0.947401,0.224891,0.517980,0.370615


In [45]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,52.194289,0.623162,20.822226,35.583901,17.462451,39.347336,20.091529,36.310602,19.671643,35.128186,...,23.215052,40.321163,19.137997,39.534826,1.962123,767.477572,90.371443,2.054723,62.023089,0.267238
1,102.260370,8.591481,21.762753,51.660389,21.694148,45.063453,21.455823,45.636309,20.793191,40.902962,...,20.466983,47.155077,15.969170,43.421453,2.324290,753.739782,90.371443,3.036032,26.839119,1.414501
2,52.194289,0.623162,21.524819,35.593048,17.457914,41.203613,21.455823,34.807780,20.156404,33.409214,...,22.260508,40.321163,19.552299,38.015512,-0.177666,767.477572,84.669430,2.054723,62.023089,-1.380087
3,52.194289,0.623162,21.524819,41.830930,23.192549,34.647423,22.292014,36.310602,22.685053,35.128186,...,21.458101,38.369921,20.295117,38.031219,9.456071,767.477572,71.809988,5.148561,39.821050,3.698667
4,73.981425,1.299753,20.003123,35.545016,16.917276,39.463946,19.978222,36.310602,19.411755,35.128186,...,22.260508,40.321163,19.130354,41.452169,-0.519158,767.477572,95.890555,4.096159,49.341081,-1.367751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,58.207425,0.623162,25.062794,46.869205,22.991525,49.228929,26.429715,44.023802,25.168084,45.169865,...,25.823914,49.862118,23.929523,47.382511,12.240487,743.970735,96.018871,1.092145,60.753961,13.255133
4930,58.207425,0.623162,20.813770,40.079015,17.457914,46.282016,20.653101,38.168104,20.793191,35.128186,...,22.260508,42.776503,19.552299,39.534826,4.649990,763.560154,90.371443,4.096159,54.365379,2.704318
4931,97.444914,6.078257,20.829493,38.212024,19.525864,37.183179,19.978222,38.168104,21.980375,35.165441,...,21.458101,36.916339,18.370548,41.452169,6.895571,749.056584,78.607813,5.163001,39.821050,2.704318
4932,97.444914,7.137608,23.002504,39.321267,22.665061,37.414303,21.455823,42.321161,21.745134,43.008454,...,20.466983,44.928520,17.701528,44.487189,2.324290,759.718885,96.002509,3.036032,34.668732,1.553524


In [46]:
df_original

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
8981,40.0,0.0,20.856667,35.363333,17.70,39.200,20.290000,36.900000,19.700000,34.200000,...,23.043750,40.251250,19.290000,39.090000,1.783333,768.850000,88.166667,2.666667,63.000000,0.000000
2755,90.0,10.0,21.890000,50.890000,21.29,45.060,21.566667,48.524583,20.356667,40.933333,...,20.600000,47.416667,16.290000,44.163333,2.733333,754.550000,89.500000,3.166667,27.333333,1.166667
9133,40.0,0.0,21.390000,35.500000,17.60,40.530,21.600000,35.200000,20.260000,33.700000,...,22.100000,39.700000,19.390000,38.372500,0.083333,766.166667,83.500000,1.833333,64.666667,-2.433333
14359,50.0,0.0,21.390000,41.033333,23.89,34.840,22.033333,36.933333,22.390000,35.236000,...,21.200000,38.090000,20.290000,38.560000,8.850000,767.800000,70.833333,5.166667,40.000000,3.716667
8876,70.0,0.0,19.890000,35.200000,16.50,40.090,20.000000,36.400000,19.230000,35.156667,...,22.100000,39.663333,19.000000,40.700000,-0.333333,769.700000,92.666667,4.000000,50.666667,-1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17496,60.0,0.0,24.790000,47.090000,22.70,49.290,26.533333,44.500000,25.142857,44.212857,...,25.790000,49.590000,23.968571,47.942857,12.300000,744.700000,99.000000,1.000000,61.000000,12.100000
9283,50.0,0.0,20.790000,40.700000,17.60,46.790,20.790000,38.230000,20.566667,35.700000,...,22.290000,41.760000,19.566667,39.200000,4.216667,764.300000,88.500000,3.833333,55.000000,2.500000
7472,60.0,0.0,21.085714,37.470000,19.89,37.270,20.200000,38.000000,22.075000,35.785000,...,21.264286,37.264286,18.600000,41.700000,6.566667,748.400000,75.333333,5.333333,40.000000,2.433333
611,80.0,10.0,23.200000,39.363333,22.39,38.260,21.133333,41.863333,21.700000,42.900000,...,20.790000,45.096667,17.600000,44.960000,2.616667,760.500000,95.166667,3.166667,37.166667,1.933333


In [47]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [48]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 64.656, 28.905, 0.589, 0.291)
Results: ('lights(t)', 4.644, 2.543, 0.642, 4720002498740756.0)
Results: ('T1(t)', 0.207, 0.168, 0.983, 0.008)
Results: ('RH_1(t)', 0.657, 0.432, 0.973, 0.011)
Results: ('T2(t)', 0.292, 0.222, 0.982, 0.011)
Results: ('RH_2(t)', 0.608, 0.465, 0.977, 0.012)
Results: ('T3(t)', 0.259, 0.207, 0.983, 0.009)
Results: ('RH_3(t)', 0.45, 0.345, 0.981, 0.009)
Results: ('T4(t)', 0.263, 0.207, 0.983, 0.01)
Results: ('RH_4(t)', 0.532, 0.424, 0.985, 0.011)
Results: ('T5(t)', 0.234, 0.171, 0.983, 0.009)
Results: ('RH_5(t)', 2.44, 1.036, 0.928, 0.019)
Results: ('T6(t)', 0.8, 0.658, 0.982, 3417081165838.907)
Results: ('RH_6(t)', 2.746, 2.214, 0.992, 0.185)
Results: ('T7(t)', 0.262, 0.203, 0.984, 0.01)
Results: ('RH_7(t)', 0.672, 0.56, 0.983, 0.016)
Results: ('T8(t)', 0.245, 0.195, 0.984, 0.009)
Results: ('RH_8(t)', 0.708, 0.568, 0.982, 0.013)
Results: ('T9(t)', 0.225, 0.171, 0.987, 0.009)
Results: ('RH_9(t)', 0.562, 0.444, 0.982, 0.011)
Results: (

,variable,rmse,mae,r2,mape
0,Appliances(t),64.656,28.905,0.589,2.910000e-01
1,lights(t),4.644,2.543,0.642,4.720002e+15
2,T1(t),0.207,0.168,0.983,8.000000e-03
3,RH_1(t),0.657,0.432,0.973,1.100000e-02
4,T2(t),0.292,0.222,0.982,1.100000e-02
5,RH_2(t),0.608,0.465,0.977,1.200000e-02
6,T3(t),0.259,0.207,0.983,9.000000e-03
7,RH_3(t),0.450,0.345,0.981,9.000000e-03
8,T4(t),0.263,0.207,0.983,1.000000e-02
9,RH_4(t),0.532,0.424,0.985,1.100000e-02
